In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from logistic_regression_yt import LogisticRegressioYT
from sklearn.model_selection import train_test_split
from preprocess import Preprocess

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

Loading data

In [3]:
raw_train_data = "../data/train.csv"
raw_test_data = "../data/test.csv"

Preprocessing

In [4]:
preproc = Preprocess()
raw_data = preproc.load_data(raw_train_data)
train_df, train_data = preproc.preprocess_data(raw_data)
train_data.shape

(44760, 20)

In [5]:
train_df.Label.value_counts(normalize=True)

0    0.785947
2    0.173280
1    0.040773
Name: Label, dtype: float64

In [6]:
X_train, y_train, X_valid, y_valid = preproc.train_valid_split(
    train_data, test_size=0.33, random_state=42)

In [7]:
X_train = preproc.normalize_data(X_train)
X_valid = preproc.normalize_data(X_valid)

Teste com uma classe só

In [8]:
y_train = (y_train == 1).astype(int)
y_valid = (y_valid == 1).astype(int)

In [9]:
y_valid

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
logistic_regression = LogisticRegressioYT(
    learning_rate=2e-5,
    epochs=1000,
    threshold=0.5,
    logging=False,
)
logistic_regression.fit(X_train, y_train)  # type: ignore
predictions = logistic_regression.predict(X_valid)  # type: ignore

accuracy = accuracy_score(y_valid, predictions)
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_valid, predictions, average="macro"
)

print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F-score:   {fscore:.3f}")

Accuracy:  0.961
Precision: 0.480
Recall:    0.500
F-score:   0.490


/Users/rafaelapinter/miniconda3/envs/dc1-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
def confusion_matrix(y_true, y_pred):
    """
    Compute confusion matrix for given true and predicted labels.

    :param y_true: Actual class labels
    :param y_pred: Predicted class labels
    :return: Confusion matrix
    """

    # Get unique class labels
    classes = np.unique(np.concatenate((y_true, y_pred)))
    num_classes = len(classes)

    # Create a mapping from class label to index
    label_to_index = {label: idx for idx, label in enumerate(classes)}

    # Initialize confusion matrix with zeros
    matrix = np.zeros((num_classes, num_classes), dtype=int)

    # Populate the confusion matrix
    for true_label, pred_label in zip(y_true, y_pred):
        matrix[label_to_index[true_label], label_to_index[pred_label]] += 1

    print("Confusion Matrix:")
    print(matrix, "\n")
    return matrix

In [18]:
confusion_matrix(y_valid, predictions)

Confusion Matrix:
[[14193     0]
 [  578     0]] 



array([[14193,     0],
       [  578,     0]])